PyCity School Analysis
February 8, 2019
Scott McEachern

In [1]:
#- Load Datasets into DataFrame
import os
import pandas as pd


# Schools DataFrame
schoolPath = os.path.join(".", "Resources", "schools_complete.csv")

school_df = pd.read_csv(schoolPath)


# Student DataFrame
studentPath = os.path.join(".", "Resources", "students_complete.csv")

student_df = pd.read_csv(studentPath)

In [8]:
#-- District Summary

#- Calculate Total Schools
totalNumSchools = school_df.shape[0]


#- Calculate Total Students
totalNumStudents = student_df.shape[0]
totalNumStudentsFormat = f"{'{:,}'.format(totalNumStudents)}"


#- Total School Budget
totalSchoolBudget = school_df["budget"].sum()
totalSchoolBudgetFormat = f"${'{:,.2f}'.format(totalSchoolBudget)}"


#- Average Math Score
aveMathScore = student_df["math_score"].mean()
aveMathScoreFormat = f"{'{:,.6f}'.format(aveMathScore)}"


#- Average Reading Score
aveReadingScore = student_df["reading_score"].mean()
aveReadingScoreFormat = f"{'{:,.6f}'.format(aveReadingScore)}"


#- Overall Passing Rate
overallAveScore = (aveMathScore + aveReadingScore) / 2
overallAveScoreFormat = f"{'{:,.6f}'.format(overallAveScore)}"


#- Percentage students with passing math score 
totalStudentsPassingMath = student_df[student_df['math_score'] >= 70].shape[0]
percentStudentsPassingMath = (totalStudentsPassingMath / totalNumStudents) * 100

percentStudentsPassingMathFormat = f"{'{:.6f}'.format(percentStudentsPassingMath)}"


#- Percentage students with passing reading score
totalStudentsPassingReading = student_df[student_df['reading_score'] >= 70].shape[0]
percentStudentsPassingReading = (totalStudentsPassingReading / totalNumStudents) * 100

percentStudentsPassingReadingFormat = f"{'{:.6f}'.format(percentStudentsPassingReading)}"


# Result Dataframe
districtSummaryResults = {
    'Total Schools': [totalNumSchools],
    'Total Students': [totalNumStudentsFormat],
    'Total Budget': [totalSchoolBudgetFormat],
    'Average Match Score': [aveMathScoreFormat],
    'Average Reading Score': [aveReadingScoreFormat],
    '% Passing Math': [percentStudentsPassingMatchFormat],
    '% Passing Reading': [percentStudentsPassingReadingFormat],
    '% Overall Passing Rate': [overallAveScoreFormat]
                }

districtSummary_df = pd.DataFrame(districtSummaryResults)

#print(percentStudentsPassingMatchFormat)

districtSummary_df

,Total Schools,Total Students,Total Budget,Average Match Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,80.431606
